In [1]:
#;.pykx.disableJupyter()

In [2]:
# https://code.kx.com/pykx/3.0/examples/jupyter-integration.html#q-first-mode
import pykx as kx
kx.util.jupyter_qfirst_enable()

PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


## KX Training: Exercises - Joins

In [5]:
trade:([]time:09:00+10*til 5;sym:`JPM`GE`JPM`IBM`GE;price:30+5?3.;size:5?20) 
order:([]time:asc `minute$08:00+5?0D02:00:00;sym:5?`JPM`GE`IBM;orderID:(5 cut 25?.Q.n),'string 5?`5; price:30+5?20.)

In [6]:
show trade:([]time:09:00+10*til 5;sym:`JPM`GE`JPM`IBM`GE;price:30+5?3.;size:5?20) 

time  sym price    size
-----------------------
09:00 JPM 31.34507 11  
09:10 GE  30.07203 11  
09:20 JPM 31.7569  16  
09:30 IBM 30.76479 7   
09:40 GE  32.3561  15  


In [7]:
show a:12 

12


### Keyed Joins

* Join the trade and order table below to see the sequence in which trades and orders happened. Matching rows should be updated and the time column should be sorted in the resulting table. 

Resulting table:

|sym| time |  px |orderID  |    price   
|---|-------|----|---------|----------
|BP | 00:05 |5  |""        |       0N    
|JPM| 00:12 |4  |""         |      0N    
|IBM| 08:30 |11 |"58528bofbc"| 46.35509
|GE | 08:58 | 0N   |"95239ceoom" |32.17365

In [ ]:
`time xasc uj[`sym xkey trade;`sym xkey order]

In [8]:
//your answer here 
`time xasc uj[`sym xkey trade; `sym xkey order]

sym| time  price    size orderID     
---| --------------------------------
GE | 08:35 48.13668 11   "63691cdnjd"
IBM| 09:08 31.03334 7    "88874kamao"
JPM| 09:55 38.50021 11   "60219ofjei"


* Using both tables below find the total ask and total bid for each sym. 

In [ ]:
quote:([]sym:`IBM`IBM`JPM`JPM;time:"u"$4 6 6.5 10;bid:.5+1 2 3 4;ask:.5+1 2 3 4)
quote2:([]sym:`IBM`IBM`JPM`JPM`IBM`JPM;time:"u"$3 4 6 6.5 8 10;bid:6?5f;ask:6?5f)

In [ ]:
quote pj `sym xkey quote2 //left table has to be keyed

Why doesn't `+` work in this example?

In [ ]:
//quote + quote2

In [ ]:
//your answer here 
sumQuote:select sum ask, sum bid by sym from quote
sumQuote2:select sum ask, sum bid by sym from quote2

pj[sumQuote;sumQuote2]

* Using `quote` and `quote2` defined above:
     * Find the matching records using ij
     * Find the matching records using ej

What do you notice ?

In [ ]:
quote ij `sym xkey quote2 //matches on records that are present in both tables
ej[`sym;quote;quote2]    //matches on all records that appear in the right hand table

In [ ]:
//your answer here
show matchingRecordsIJ:ij[quote;`sym xkey quote2]
show matchingRecordsEJ:ej[`sym;quote;quote2:`sym xkey quote2]

* Given the following tables regarding race times, take today's race times and integrate them with the fastest time from each of the last ten races for each participant so all information can be accessed from one table.               

Resulting table:

|name|  time |    bestTime|
|------|--------|---------|
|tom|   11.19638| 11.08389|
|dick|  11.25855| 11.19599|
|harry| 11.25799| 11.08124|

In [ ]:
race lj select bestTime:min time by name from(uj/)(tom;dick;harry)

In [11]:
race:([]name:`tom`dick`harry;time:11+3?0.5)
tom:([]name:`tom;time:11+10?1f)
dick:([]name:`dick;time:11+10?1f)
harry:([]name:`harry;time:11+10?1f)
show unionTable:(uj/)(tom;dick;harry)
race lj select bestTime:min time by name from unionTable


name  time     bestTime
-----------------------
tom   11.17798 11.46237
dick  11.38813 11.04623
harry 11.10327 11.33882
name time    
-------------
tom  11.5608 
tom  11.99956
tom  11.95737
tom  11.46237
tom  11.9641 
tom  11.92842
tom  11.77645
tom  11.86279
tom  11.46715
tom  11.47473
dick 11.04623
dick 11.93329
dick 11.20727
dick 11.72628
dick 11.68804
dick 11.34077
dick 11.91786
dick 11.06613
dick 11.77249
dick 11.7098 
..


* Use iterators with joins to combine a series of tables with one common column 
                    

In [ ]:
(lj/)[t;1!/:(t1;t2;t3)] 

In [12]:
t:`a xkey t:([]a:1 2 3;b:4 5 6)
t1:`a xkey t1:([]a:1 2 3;c:7 8 9)
t2:`a xkey t2:([]a:1 2 3;d:10 11 12)
t3:`a xkey t3:([]a:1 2 3;e:13 14 15)

(lj/)(t;t1;t2;t3)

a| b c d  e 
-| ---------
1| 4 7 10 13
2| 5 8 11 14
3| 6 9 12 15


In [ ]:
//your answer here 

### Bitemporal joins 

* Below is table containing a record of changes in the telephone numbers of Tom and Bob during the last year:

```phoneNum:([Name:`Tom`Bob`Tom`Bob;Date:2014.06.01 2014.06.01 2015.01.01 2015.06.01] phNum:336699 123456 999999 778899)```

Using phoneNum produce a table that shows Bob’s telephone number as at the current dates: `2013.06.02`,`2015.01.01`, `2015.01.02` and today.

In [ ]:
phoneNum:([Name:`Tom`Bob`Tom`Bob;
                    Date:2014.06.01 2014.06.01 2015.01.01 2015.06.01]
                    phNum:336699 123456 999999 778899)
show p:([]Name:4#`Bob;Date:(2013.06.02;2015.01.01;2015.01.02;.z.d))  //built our rack of times 
aj[`Name`Date;p;0!phoneNum] //need the name to be the same, otherwise we'll get Toms results 

In [13]:
//your answer here 
show phoneNum:([Name:`Tom`Bob`Tom`Bob;Date:2014.06.01 2014.06.01 2015.01.01 2015.06.01] phNum:336699 123456 999999 778899)

Name Date      | phNum 
---------------| ------
Tom  2014.06.01| 336699
Bob  2014.06.01| 123456
Tom  2015.01.01| 999999
Bob  2015.06.01| 778899


In [14]:
show p:([]Name:4#`Bob;Date:(2013.06.02;2015.01.01;2015.01.02;.z.d))

Name Date      
---------------
Bob  2013.06.02
Bob  2015.01.01
Bob  2015.01.02
Bob  2025.03.15


In [16]:
aj[`Name`Date;p;phoneNum]

Name Date       phNum 
----------------------
Bob  2013.06.02       
Bob  2015.01.01 123456
Bob  2015.01.02 123456
Bob  2025.03.15 778899


* Use a series of joins to merge the three tables to be able to access data regarding each sym from one table. 

Resulting table:

|sym|  time|  price| bid|      ask|      location|   
|----|-------|--------|-----|-----------|-----------|
|AAPL| 09:00| 50|    78.33686| 27.82122| "Cupertino"|
|JPM | 09:02| 100|   61.08817| 15.08133| "New York"| 
|TSLA| 09:05| 150|   1.392076| 94.41671| "Palo Alto"|

In [ ]:
aj[`sym`time;stockPrice;stockTrades]lj 1!select sym:company,location from stocks

In [17]:
show stocks:([]company:`AAPL`TSLA`JPM;location:("Cupertino";"Palo Alto";"New York"))


company location   
-------------------
AAPL    "Cupertino"
TSLA    "Palo Alto"
JPM     "New York" 


In [19]:
show stockPrice:([]sym:`AAPL`JPM`TSLA;time:09:00 09:02 09:05;price:50 100 150)

sym  time  price
----------------
AAPL 09:00 50   
JPM  09:02 100  
TSLA 09:05 150  


In [20]:
show stockTrades:([]sym:`AAPL`AAPL`JPM`AAPL`JPM`JPM`TSLA;time:09:00 09:01 09:02 09:03 09:04 09:04 09:05;bid:7?100f;ask:7?100f)

sym  time  bid      ask     
----------------------------
AAPL 09:00 76.7256  95.11239
AAPL 09:01 26.70152 87.13601
JPM  09:02 65.6726  59.11021
AAPL 09:03 54.40371 68.15041
JPM  09:04 86.93354 82.8788 
JPM  09:04 33.58802 59.3962 
TSLA 09:05 50.74213 14.46023


In [28]:
//your answer here 
stocks:`sym xcol stocks
lj[stockTrades;`sym xkey stocks]

sym  time  bid      ask      location   
----------------------------------------
AAPL 09:00 76.7256  95.11239 "Cupertino"
AAPL 09:01 26.70152 87.13601 "Cupertino"
JPM  09:02 65.6726  59.11021 "New York" 
AAPL 09:03 54.40371 68.15041 "Cupertino"
JPM  09:04 86.93354 82.8788  "New York" 
JPM  09:04 33.58802 59.3962  "New York" 
TSLA 09:05 50.74213 14.46023 "Palo Alto"


In [30]:
//Final Answer
aj[`time;stockPrice;lj[stockTrades;`sym xkey stocks]]

sym  time  price bid      ask      location   
----------------------------------------------
AAPL 09:00 50    76.7256  95.11239 "Cupertino"
JPM  09:02 100   65.6726  59.11021 "New York" 
TSLA 09:05 150   50.74213 14.46023 "Palo Alto"


* Write a function: **getPrevailingQuote**
    - Returns a join of the trade and quote table, which shows each trade record with the prevailing quote (on any exchange) at the time of each trade
    - Quotes should be specific to the sym
    - Assumes global table trade and quote 

In [ ]:
getPrevailingQuote:{aj[`sym`time;get `trade;get `quote]}

In [34]:
getPrevailingQuote:{aj[`sym`time;trade;quote]}

Example of Code:

    trade:([]sym:`BP`IBM`IBM`JPM;time:"u"$4.5 6.2 9.8 12;px:5 11 8 4f)                                         
    
    quote:([]sym:`IBM`IBM`JPM`JPM;time:"u"$4 6 6.5 10;bid:.5+1 2 3 4;ask:.5+1 2 3 4) 
    
    getPrevailingQuote[]
    
    sym time  px bid ask     
    --------------------     
    BP  00:05 5              
    IBM 00:06 11 2.5 2.5     
    IBM 00:10 8  2.5 2.5     
    JPM 00:12 4  4.5 4.5

In [35]:
trade:([]sym:`BP`IBM`IBM`JPM;time:"u"$4.5 6.2 9.8 12;px:5 11 8 4f)                                         
quote:([]sym:`IBM`IBM`JPM`JPM;time:"u"$4 6 6.5 10;bid:.5+1 2 3 4;ask:.5+1 2 3 4) 

In [36]:
getPrevailingQuote[]      //test output

sym time  px bid ask
--------------------
BP  00:05 5         
IBM 00:06 11 2.5 2.5
IBM 00:10 8  2.5 2.5
JPM 00:12 4  4.5 4.5




* Write a function: **getPrevailingLocalQuote**
    - Returns a join of the trade and quote table, which shows each trade record with the prevailing quote at the time of each trade on that exchange.

In [ ]:
getPrevailingLocalQuote:{select from aj[`sym`ex`time;get `trade;get `quote]}

In [37]:
getPrevailingLocalQuote:{aj[`ex`sym`time;trade;quote]}

Example of Code:

    trade:([]sym:`BP`IBM`IBM`JPM;time:"u"$4.5 6.2 9.8 12;px:5 11 8 4f';ex:`A`A`A`B)                                         
    
    quote:([]sym:`IBM`IBM`JPM`JPM;time:"u"$4 6 6.5 10;bid:.5+1 2 3 4;ask:.5+1 2 3 4;ex:`A`A`A`A) 
    
    getPrevailingLocalQuote[]
    
    sym time  px ex bid ask
    -----------------------
    BP  00:05 5  A        
    IBM 00:06 11 A  2.5 2.5
    IBM 00:10 8  A  2.5 2.5
    JPM 00:12 4  B

In [38]:
show trade:([]sym:`BP`IBM`IBM`JPM;time:"u"$4.5 6.2 9.8 12;px:5 11 8 4f;ex:`A`A`A`B)                                         
show quote:([]sym:`IBM`IBM`JPM`JPM;time:"u"$4 6 6.5 10;bid:.5+1 2 3 4;ask:.5+1 2 3 4;ex:`A`A`A`A) 

sym time  px ex
---------------
BP  00:05 5  A 
IBM 00:06 11 A 
IBM 00:10 8  A 
JPM 00:12 4  B 
sym time  bid ask ex
--------------------
IBM 00:04 1.5 1.5 A 
IBM 00:06 2.5 2.5 A 
JPM 00:07 3.5 3.5 A 
JPM 00:10 4.5 4.5 A 


In [39]:
getPrevailingLocalQuote[]  //test output 

sym time  px ex bid ask
-----------------------
BP  00:05 5  A         
IBM 00:06 11 A  2.5 2.5
IBM 00:10 8  A  2.5 2.5
JPM 00:12 4  B         


* Use an asof join to return the prevailing quote for every sym at the beginning of every hour

In [ ]:
/z will contain the price for each traded symbol as of the start of each hour 
z:([]sym:`symbol$();time:`time$())

/the function f will be used to initially populate z
f:{`z insert (x;y)}

/initially populate z
(exec distinct sym from select sym from trade)f/:\:(exec distinct `time$60*60*1000*time.hh from select time from trade)

/use asof join to insert the prevailing quote as of the start of each hour
aj[`sym`time;z;update qtime:time from select from quote]

In [40]:
//your answer here
show z:([]sym:`symbol$();time:`time$())

sym time
--------


In [41]:
f:{`z insert (x;y)}

In [46]:
(exec distinct sym from select sym from trade)f/:\:(exec distinct `time$60*60*1000*time.hh from select time from trade) 
// need to figure out what is going on

3
4
5


In [45]:
//aj[`sym`time;z;update qtime:time from select from quote]  has an error, no idea why

In [47]:
z

sym time        
----------------
BP  00:00:00.000
IBM 00:00:00.000
JPM 00:00:00.000
BP  00:00:00.000
IBM 00:00:00.000
JPM 00:00:00.000


* Construct a 1 second window join between trade and quote. This window join should calculate the max ask and  min bid for that window while joining the trade and quote table.

In [ ]:
w:trade[`time]+/: -1 1 //specifing one second before and after the trade

In [ ]:
wj[w;`sym`time;trade;(quote;(max;`ask);(min;`bid))]

In [51]:
//your answer here
show windows:(trade[`time]-1;trade[`time]+1)
quote

sym time  bid ask ex
--------------------
IBM 00:04 1.5 1.5 A 
IBM 00:06 2.5 2.5 A 
JPM 00:07 3.5 3.5 A 
JPM 00:10 4.5 4.5 A 
00:04 00:05 00:09 00:11
00:06 00:07 00:11 00:13


In [52]:
wj[windows;`sym`time;trade;(quote;(max;`ask);(min;`bid))]

sym time  px ex ask bid
-----------------------
BP  00:05 5  A  -0w 0w 
IBM 00:06 11 A  2.5 1.5
IBM 00:10 8  A  2.5 2.5
JPM 00:12 4  B  4.5 4.5


* Find the average trade price for the two seconds before each trade using a window join

In [59]:
show window:trade[`time]+/:-1 0 //specifing 1 second window
wj1[window;`sym`time;trade;(trade;(avg;`px))] //price column has been overwritten 

sym time  px ex
---------------
BP  00:05 5  A 
IBM 00:06 11 A 
IBM 00:10 8  A 
JPM 00:12 4  B 
00:04 00:05 00:09 00:11
00:05 00:06 00:10 00:12


In [ ]:
wj1[window;`sym`time;trade;(`sym`time`avgPx`ex xcol trade;(avg;`avgPx))] //renaming px column so we can easily compare

In [53]:
//your answer here
trade

sym time  px ex
---------------
BP  00:05 5  A 
IBM 00:06 11 A 
IBM 00:10 8  A 
JPM 00:12 4  B 


In [54]:
quote

sym time  bid ask ex
--------------------
IBM 00:04 1.5 1.5 A 
IBM 00:06 2.5 2.5 A 
JPM 00:07 3.5 3.5 A 
JPM 00:10 4.5 4.5 A 


In [62]:
show newwindows:(trade[`time]-2;trade[`time])


00:03 00:04 00:08 00:10
00:05 00:06 00:10 00:12


In [63]:
wj[newwindows;`sym`time;trade;(trade;(avg;`px))]

sym time  px  ex
----------------
BP  00:05 5   A 
IBM 00:06 11  A 
IBM 00:10 9.5 A 
JPM 00:12 4   B 
